In [10]:
from datetime import datetime
import os
import cv2
path = 'cars-and-trucks/'

In [11]:
def get_filename():
    now = datetime.now()
    time_stamp = now.strftime("%m-%d-%Y %H:%M:%S")
    file_name = time_stamp + '.mp4'
    return (os.path.join(path, file_name))

In [12]:
def speed_in_mph(x1,x2,f1,f2):
    if x1 == 0:
        return(None)
#     print("x1: ",x1, "x2: ",x2, "f1: ",f1, "f2: ",f2)
    speed_of_car = 0
    pixels_traveled = abs(x1 - x2) # first to last notice of a car
#     print('in speed_in_mph')
#     print('pixels traveled: ',pixels_traveled)
    frames_with_cars = abs(f1 - f2) # from containing the first and last notice of a car
#     print('frames with cars: ', frames_with_cars)
    if frames_with_cars > 0:
        time_elapsed = frames_with_cars / 30 # number of seconds from first frame with car to last frame with car
#         print ('Time elapsed: ',time_elapsed,' seconds')
        field_traveled = pixels_traveled / 320  # what portion of the field of view was the car noticed
#         print ('portion of the view traveled: ',field_traveled*100,'%')
        distance_traveled = field_traveled * 65 # feet of travel - averaged of near and far lanes 50 & 45' respectively
#         print ('Distance traveled: ',distance_traveled,' in feet')
        feet_per_second = distance_traveled / time_elapsed
#         print('Feet per second: ',feet_per_second)
        speed_of_car = feet_per_second * 0.6818181818 # convert feet per second the miles per hour
#         print('Speed of car: ',speed_of_car)
    return (speed_of_car)

In [13]:
def found_a_car(location):
    big_x = 0
    little_x = 320
    frame_big_x = 0
    frame_little_x = 0
    speed = 0
    p = 0
#     print('In found_a_car')
    for (x,y,w,h,frames) in location:
            p = x + w / 2 # a point midway through the rectangle 
            if p>big_x:
                big_x = p
                frame_big_x = frames
            if p<little_x:
                little_x = p
                frame_little_x = frames
    if big_x != 0:
        speed = round(speed_in_mph(big_x,little_x,frame_big_x,frame_little_x),3) # speed not set
    return(speed)

In [14]:
def look_for_cars():
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
            print('Error - could not get the camera opened')
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 320) # supported 160x90 or 320x180
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 180)
    font = cv2.FONT_HERSHEY_SIMPLEX
    car_cascade = cv2.CascadeClassifier('cars.xml')
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    recording = False
    frames_with_cars = 0
    frames_since_recording_started = 0
    finished = False
    location = []
    stop_looking = False
    while not finished:
        # reads frame from a video
        ret, frame = cap.read() # does every frame get sent to the file?
        if ret: # if you received a frame, then proceed
            if recording:
                frames_since_recording_started += 1
            if not recording:
                cv2.imshow('Car Detection', frame)
            # convert to gray scale of each frame before doing the cascade
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            # Detects cars of different sizes in the input image
            cars = car_cascade.detectMultiScale(gray, 1.1, 2)
            # Draw a rectangle of each car in the frame
            for (x,y,w,h) in cars:
                cv2.rectangle(frame,(x,y),(x+w,y+h),(0,0,255),2)
                location.append([x,y,w,h,frames_with_cars])
                # Display frame in a window 
                cv2.imshow('Car Detection', frame)
                frames_with_cars += 1
                if not recording:
                    video_file = get_filename()
                    print(video_file,end='')
                    writer = cv2.VideoWriter(video_file, 
                                     cv2.VideoWriter_fourcc(*'mp4v'),
                                     30, (width, height))
                    recording = True
            if recording:
                writer.write(frame)
            # If you have at least 60 framend=''es since you first detected a car
            finished = (frames_since_recording_started - frames_with_cars > 20) 
        if cv2.waitKey(1) == 27:
            finished = True
            how_fast = 0
            stop_looking = True
            break
        if recording:
            how_fast = found_a_car(location)
            # start here put the speed in the frame
            cv2.putText(frame,text='{:.2f}'.format(how_fast) + 'mph avg speed @' + video_file,
                            org=(5,150),fontFace=font,
                            fontScale=0.25,color=(0,0,0),thickness=1,
                            lineType=cv2.LINE_AA)
                
    print(' estimated speed: ',how_fast," mph")
    
    cap.release()
    if recording:
        hold_frame = 90
        while hold_frame > 0:
            writer.write(frame)
            hold_frame -= 1
        writer.release()
        if how_fast < 35: # delete the file if the car is going less than 35 mph mean speed
            if os.path.isfile(video_file):
                os.remove(video_file)
            else: 
                print("Error: %s file not found" % video_file)
    cv2.destroyAllWindows()
#     print('frames_with_cars: ',frames_with_cars)
#     print('frames_since_recording_started: ', frames_since_recording_started)
    return(finished, stop_looking)

In [ ]:
video_count = 0
while (video_count < 500):
    recoding_ended, interrupt = look_for_cars()
    if interrupt:
        break
    video_count += 1

cars-and-trucks/08-17-2021 14:13:41.mp4 estimated speed:  23.981  mph
cars-and-trucks/08-17-2021 14:14:51.mp4 estimated speed:  27.699  mph
cars-and-trucks/08-17-2021 14:15:12.mp4 estimated speed:  32.645  mph
cars-and-trucks/08-17-2021 14:16:10.mp4 estimated speed:  25.944  mph
cars-and-trucks/08-17-2021 14:18:26.mp4 estimated speed:  24.006  mph
cars-and-trucks/08-17-2021 14:18:52.mp4 estimated speed:  29.695  mph
cars-and-trucks/08-17-2021 14:18:54.mp4 estimated speed:  21.12  mph
cars-and-trucks/08-17-2021 14:19:32.mp4 estimated speed:  23.414  mph
cars-and-trucks/08-17-2021 14:20:24.mp4 estimated speed:  28.045  mph
cars-and-trucks/08-17-2021 14:20:38.mp4 estimated speed:  30.123  mph
cars-and-trucks/08-17-2021 14:21:33.mp4 estimated speed:  31.667  mph
cars-and-trucks/08-17-2021 14:23:00.mp4 estimated speed:  33.483  mph
cars-and-trucks/08-17-2021 14:25:44.mp4 estimated speed:  29.776  mph
cars-and-trucks/08-17-2021 14:28:50.mp4 estimated speed:  30.805  mph
cars-and-trucks/08-17

In [ ]:
#!v4l2-ctl --list-devices 